<h3>Import Necessary Libarary</h3>

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset, Dataset
import torch
from transformers import Trainer, TrainingArguments

<h3>Load Dataset</h3>

In [8]:
# Load the data from the file
file_path = './enhanced_fruits_dataset.txt'
with open(file_path, 'r') as file:
    data = file.readlines()

# Strip any leading/trailing whitespace characters
data = [line.strip() for line in data]

# Convert data to Hugging Face Dataset
dataset = Dataset.from_dict({'text': data})
dataset

Dataset({
    features: ['text'],
    num_rows: 66840
})

<h3>Tokenize Dataset</h3>

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized_output = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
    tokenized_output['labels'] = tokenized_output['input_ids'].copy()  # Copy input_ids to labels
    return tokenized_output

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask','labels'])
tokenized_datasets

Map: 100%|██████████| 66840/66840 [00:21<00:00, 3140.92 examples/s]


Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 66840
})

<h3>Training dataset</h3>

In [10]:
# Split the dataset into train and test sets
train_size = int(0.9 * len(tokenized_datasets))
test_size = len(tokenized_datasets) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(tokenized_datasets, [train_size, test_size])

<h3>Load GPT-2 Model</h3>

In [11]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

<h3>Train Model</h3>

In [12]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True,       # overwrite the content of the output directory
    num_train_epochs=5,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    save_steps=5000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()


                                         
  0%|          | 0/37600 [08:46<?, ?it/s]             

{'loss': 9.6101, 'grad_norm': 206.12342834472656, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


                                         
  0%|          | 0/37600 [12:18<?, ?it/s]             

{'loss': 8.6267, 'grad_norm': 190.94349670410156, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


                                         
  0%|          | 0/37600 [15:48<?, ?it/s]             

...

{'loss': 0.0086, 'grad_norm': 0.2381521612405777, 'learning_rate': 4.8787061994609166e-05, 'epoch': 0.19}


                                           
  0%|          | 0/37600 [8:16:44<?, ?it/s]               

{'loss': 0.0083, 'grad_norm': 0.21443569660186768, 'learning_rate': 4.877358490566038e-05, 'epoch': 0.19}


                                           
  0%|          | 0/37600 [8:20:12<?, ?it/s]               

{'loss': 0.0083, 'grad_norm': 0.2650510370731354, 'learning_rate': 4.8760107816711594e-05, 'epoch': 0.19}
